In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_062_PIANO062_MID--AUDIO-split_07-07-17_Piano-e_2-07_wav--2.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_060_PIANO060_MID--AUDIO-split_07-07-17_Piano-e_2-04_wav--4.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_045_PIANO045_MID--AUDIO-split_07-06-17_Piano-e_2-01_wav--2.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_081_PIANO081_MID--AUDIO-split_07-09-17_Piano-e_2_-02_wav--1.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_058_PIANO058_MID--AUDIO-split_07-07-17_Piano-e_2-02_wav--4.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_042_PIANO042_MID--AUDIO-split_07-06-17_Piano-e_1-02_wav--2.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/MIDI-Unprocessed_062_PIANO062_MID--AUDIO-split_07-07-17_Piano-e_2-07_wav--3.midi
/kaggle/input/the-maestro-dataset/maestro-v3.0.0/2017/

In [3]:
# use this if tensorflow library gives any error or else comment it 
!pip install --upgrade --force-reinstall tensorflow==2.17.0 protobuf==3.20.*

In [2]:
import tensorflow as tf
print("GPUs available:", tf.config.list_physical_devices('GPU'))

2025-11-13 08:35:30.310462: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763022930.735547      48 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763022930.842431      48 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPUs available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [4]:
import glob
import pickle
import numpy as np
from tqdm import tqdm
from music21 import converter, note, chord
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint

#  Paths

In [5]:
PKL_PATH = "/kaggle/input/extracted-music-data/music_data.pkl"
MIDI_DIR = "/kaggle/input/the-maestro-dataset/maestro-v3.0.0"

#  Step 1: Load or Create Preprocessed Data

In [6]:
if os.path.exists(PKL_PATH):
    print(f" Found existing preprocessed data at:\n{PKL_PATH}")
    with open(PKL_PATH, "rb") as f:
        data = pickle.load(f)

    notes = data["notes"]
    unique_notes = data["unique_notes"]
    note_to_int = data["note_to_int"]
    X = data["X"]
    y = data["y"]

else:
    print("No preprocessed data found. Extracting from MIDI files...")

    midi_files = glob.glob(os.path.join(MIDI_DIR, "**", "*.midi"), recursive=True)
    print(f"Found {len(midi_files)} MIDI files.")
    notes = []

    for file in tqdm(midi_files, desc="Processing MIDI files", unit="file"):
        try:
            midi = converter.parse(file)
            for element in midi.recurse():
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))
        except Exception as e:
            print(f"Skipping {file}: {e}")

    print(f"\nTotal notes extracted: {len(notes)}")
    print("Sample notes:", notes[:10])

    # Prepare sequences
    sequence_length = 100
    unique_notes = sorted(list(set(notes)))
    note_to_int = {n: i for i, n in enumerate(unique_notes)}

    network_input = []
    network_output = []

    for i in range(0, len(notes) - sequence_length):
        seq_in = notes[i:i + sequence_length]
        seq_out = notes[i + sequence_length]
        network_input.append([note_to_int[n] for n in seq_in])
        network_output.append(note_to_int[seq_out])

    X = np.reshape(network_input, (len(network_input), sequence_length, 1)) / float(len(unique_notes))
    y = np.array(network_output)

    data_to_save = {
        "notes": notes,
        "unique_notes": unique_notes,
        "note_to_int": note_to_int,
        "X": X,
        "y": y
    }

    with open(PKL_PATH, "wb") as f:
        pickle.dump(data_to_save, f)
    print(f" Saved preprocessed data to: {PKL_PATH}")

 Found existing preprocessed data at:
/kaggle/input/extracted-music-data/music_data.pkl


#  Step 2: Define Model

In [7]:
model = Sequential([
    LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
    Dropout(0.3),
    LSTM(256),
    Dense(256, activation='relu'),
    Dense(len(unique_notes), activation='softmax')
])

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.001)
)

I0000 00:00:1763022967.680373      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1763022967.681025      48 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#  Step 3: Train Model

In [ ]:
checkpoint = ModelCheckpoint(
    "checkpoints/music_epoch_{epoch:02d}.keras",
    save_weights_only=False,
    save_freq='epoch'
)

print("\nStarting model training...\n")
history = model.fit(X, y, epochs=10, batch_size=64, verbose=1, callbacks=[checkpoint])


Starting model training...

Epoch 1/10


I0000 00:00:1763022973.602635     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


28382/28382 ━━━━━━━━━━━━━━━━━━━━ 544s 19ms/step - loss: 5.3721
Epoch 2/10
28382/28382 ━━━━━━━━━━━━━━━━━━━━ 546s 19ms/step - loss: 5.2530
Epoch 3/10
28382/28382 ━━━━━━━━━━━━━━━━━━━━ 543s 19ms/step - loss: 5.1970
Epoch 4/10
16009/28382 ━━━━━━━━━━━━━━━━━━━━ 4:00 19ms/step - loss: 5.1664

#  Step 4: Save Model

In [ ]:
model.save("music_gen_model.keras")
print("\n Model saved as 'music_gen_model.keras'")